In [ ]:
import json
import random
from pronto import Ontology, Definition

class ImpatientVocab():
    def __init__(self) -> None:
        self.used_colors: list[str] = []
        self.impatient_json: list[dict] = []
        self.impatient_onto: Ontology = None
        self.list_of_terms: list[str] = []

    def load_json(self, path: str) -> list[dict]:
        self.impatient_json = json.load(open(path, "r"))
        return self.impatient_json
    
    def load_ontology(self, path: str) -> Ontology:
        self.impatient_onto = Ontology(path)
        return self.impatient_onto
    
    def json_to_onto(self) -> Ontology:
        self.impatient_onto = Ontology()
        for term in self.impatient_json:
            added_term = self.impatient_onto.create_term(term["id"].replace("_", ":"))
            added_term.name = term["text"]
            for syn in term["data"]["synonymes"].split(","):
                if syn != "":
                    added_term.add_synonym(syn, scope="EXACT")
            if term["data"]["description"] != "":
                added_term.definition = Definition(term["data"]["description"])
            if term["parent"] != "#":
                added_term.superclasses().add(self.impatient_onto[term["parent"].replace("_", ":")])
        
            self.list_of_terms.append(added_term)
        return self.impatient_onto
    
    def onto_to_json(self) -> list[dict]:
        self.impatient_json = []
        index = 0
        for term in self.impatient_onto.terms():
            relationships = []
            for rel in term.superclasses():
                relationships.append(rel.id)
            relationships.pop(0)
            self.impatient_json.append(
                {
                    "id": term.id.replace("_", ":"),
                    "text": term.name,
                    "icon": True,
                    "data": {
                        "description": term.definition if term.definition is not None else "",
                        "synonymes": ",".join([syn.description for syn in term.synonyms]),
                        "phenotype_datamined": "",
                        "gene_datamined": "",
                        "alternative_language": term.name,
                        "correlates_with": "",
                        "image_annotation": True if index == 0 else False,
                        "hex_color": self._generate_hex_color(),
                        "hpo_datamined": "",
                    },
                    "parent": relationships[0].replace("_", ":") if relationships != [] else "#"
                }
            )
            index += 1
        return self.impatient_json
    
    def _generate_hex_color(self):
        while True:
            # Generate a random hex color
            color = "#{:06x}".format(random.randint(0, 0xFFFFFF))
            # Check if the color has already been used
            if color not in self.used_colors:
                # Add the color to the list of used colors and return it
                self.used_colors.append(color)
                return color
        
    def dump_onto(self, path: str) -> None:
        with open(path, "wb") as f:
            self.impatient_onto.dump(f, format="obo")

    def dump_json(self, path: str) -> None:
        with open(path, "w") as f:
            json.dump(self.impatient_json, f, indent=2)

In [ ]:
my_onto = ImpatientVocab()
my_onto.load_json("ontology.json.demo")
my_onto.json_to_onto()
my_onto.dump_onto("ontology_imp.obo")

In [ ]:
my_onto = ImpatientVocab()
my_onto.load_ontology("goslim_agr.obo")
my_onto.onto_to_json()
my_onto.dump_json("obo_to_json_GO.json")

In [ ]:
my_onto = ImpatientVocab()
my_onto.load_ontology("ontology_imp.obo")
my_onto.onto_to_json()
my_onto.dump_json("obo_to_json_IMP.json")

In [ ]:
my_onto = ImpatientVocab()
my_onto.load_ontology("hp.owl")
my_onto.onto_to_json()
my_onto.dump_json("obo_to_json_HPO.json")